In [6]:
# Code for 1b
import numpy as np

# Crout LU decomposition + forward/back substitution

def crout_lu(A: np.ndarray):
    A = A.astype(float)
    n = A.shape[0]
    L = np.zeros((n, n), dtype=float)
    U = np.zeros((n, n), dtype=float)

    for i in range(n):
        U[i, i] = 1.0

    for j in range(n):
        for i in range(j, n):
            s = 0.0
            for k in range(j):
                s += L[i, k] * U[k, j]
            L[i, j] = A[i, j] - s

        if abs(L[j, j]) < 1e-14:
            raise ZeroDivisionError(f"Zero pivot at L[{j},{j}] = {L[j,j]}")

        for i in range(j + 1, n):
            s = 0.0
            for k in range(j):
                s += L[j, k] * U[k, i]
            U[j, i] = (A[j, i] - s) / L[j, j]

    return L, U

def forward_substitution(L: np.ndarray, b: np.ndarray):
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    for i in range(n):
        s = 0.0
        for k in range(i):
            s += L[i, k] * y[k]
        y[i] = (b[i] - s) / L[i, i]
    return y

def back_substitution(U: np.ndarray, y: np.ndarray):
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        s = 0.0
        for k in range(i + 1, n):
            s += U[i, k] * x[k]
        x[i] = (y[i] - s) / U[i, i]
    return x

# Build A and b from the equation (5-point stencil)
def assemble_laplace_Ab(m, n, T_left, T_right, T_bottom, T_top):
    """
    Assemble A x = b for Laplace equation on a unit square
    using a 5-point stencil on an m×n interior grid.

    Unknowns are interior nodes (i=1..m, j=1..n).
    We map (i,j) -> p index by: p = (j-1)*m + (i-1)
    """

    N = m * n
    A = np.zeros((N, N), dtype=float)
    b = np.zeros(N, dtype=float)

    def idx(i, j):
        """Map interior (i,j) to 1D index p."""
        return (j - 1) * m + (i - 1)

    # Loop over each interior node (i,j)
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            p = idx(i, j)

            # Center coefficient from 4u_ij - neighbors = boundary terms
            A[p, p] = 4.0

            # Left neighbor (i-1, j)
            if i - 1 >= 1:  # interior neighbor
                A[p, idx(i - 1, j)] = -1.0
            else:           # boundary x=0
                b[p] += T_left

            # Right neighbor (i+1, j)
            if i + 1 <= m:
                A[p, idx(i + 1, j)] = -1.0
            else:           # boundary x=1
                b[p] += T_right

            # Bottom neighbor (i, j-1)
            if j - 1 >= 1:
                A[p, idx(i, j - 1)] = -1.0
            else:           # boundary y=0
                b[p] += T_bottom

            # Top neighbor (i, j+1)
            if j + 1 <= n:
                A[p, idx(i, j + 1)] = -1.0
            else:           # boundary y=1
                b[p] += T_top

    return A, b

# Main: boundary values + solve with LU
if __name__ == "__main__":
    # Boundary Conditions (°C)
    T_left   = 50.0
    T_right  = 25.0
    T_bottom = 0.0
    T_top    = 75.0

    # 2×2 interior grid (same as Part (a))
    m, n = 2, 2

    # Assemble A and b from the Laplace equation (no manual A)
    A, b = assemble_laplace_Ab(m, n, T_left, T_right, T_bottom, T_top)

    # Solve using Crout LU
    L, U = crout_lu(A)
    y = forward_substitution(L, b)
    x = back_substitution(U, y)

    # x ordering corresponds to (i,j) in the mapping:
    # p=0:(1,1)=T11, p=1:(2,1)=T21, p=2:(1,2)=T12, p=3:(2,2)=T22
    T11, T21, T12, T22 = x

    np.set_printoptions(precision=6, suppress=True)

    print("\n1(b) LU (Crout) Solver Output (A built from equation)\n")
    print("A =\n", A)
    print("\nb = ", b)

    print("\nInterior temperatures (°C):")
    print(f"T11 = {T11:.6f}")
    print(f"T21 = {T21:.6f}")
    print(f"T12 = {T12:.6f}")
    print(f"T22 = {T22:.6f}")


1(b) LU (Crout) Solver Output (A built from equation)

A =
 [[ 4. -1. -1.  0.]
 [-1.  4.  0. -1.]
 [-1.  0.  4. -1.]
 [ 0. -1. -1.  4.]]

b =  [ 50.  25. 125. 100.]

Interior temperatures (°C):
T11 = 31.250000
T21 = 25.000000
T12 = 50.000000
T22 = 43.750000


In [ ]:
# Code for 1c
import numpy as np

# Liebmann (Gauss–Seidel) Iteration with Over-Relaxation (SOR)
# for 2D Laplace Equation

# Boundary Conditions (°C)
T_left   = 50.0
T_right  = 25.0
T_bottom = 0.0
T_top    = 75.0

# Interior grid size (same system as Part a)
m, n = 2, 2

# Over-relaxation parameter and tolerance
lam = 1.5          # 1 ≤ λ ≤ 2
tol = 1e-6
max_iter = 10000

# Grid including boundaries
# Size: (n+2) × (m+2)
u = np.zeros((n+2, m+2))

# Apply boundary conditions
u[:, 0]  = T_left
u[:, -1] = T_right
u[0, :]  = T_bottom
u[-1, :] = T_top

# Initial guess: average of boundaries
u[1:-1, 1:-1] = (T_left + T_right + T_bottom + T_top) / 4

# Liebmann Iteration (Gauss–Seidel + SOR)
for k in range(1, max_iter + 1):
    max_diff = 0.0

    for j in range(1, n + 1):
        for i in range(1, m + 1):

            u_star = 0.25 * (
                u[j, i+1] + u[j, i-1] +
                u[j+1, i] + u[j-1, i]
            )

            u_new = lam * u_star + (1 - lam) * u[j, i]
            diff = abs(u_new - u[j, i])

            u[j, i] = u_new
            max_diff = max(max_diff, diff)

    if max_diff < tol:
        print(f"Converged in {k} iterations.")
        break

# Extract interior values
T11 = u[1, 1]
T21 = u[1, 2]
T12 = u[2, 1]
T22 = u[2, 2]

print("\n1(c) Liebmann Iteration with Over-Relaxation\n")
print(f"T11 = {T11:.6f}")
print(f"T21 = {T21:.6f}")
print(f"T12 = {T12:.6f}")
print(f"T22 = {T22:.6f}")

Converged in 26 iterations.

(c) Liebmann Iteration with Over-Relaxation

T11 = 31.250000
T21 = 25.000000
T12 = 50.000000
T22 = 43.750000


In [ ]:
# Code for 3c
import numpy as np
import math

# Crout LU + forward/back solve
def crout_lu(A: np.ndarray):
    A = A.astype(float)
    n = A.shape[0]
    L = np.zeros((n, n), dtype=float)
    U = np.zeros((n, n), dtype=float)

    # Crout: diag(U) = 1
    for i in range(n):
        U[i, i] = 1.0

    for j in range(n):
        # Column j of L
        for i in range(j, n):
            s = 0.0
            for k in range(j):
                s += L[i, k] * U[k, j]
            L[i, j] = A[i, j] - s

        if abs(L[j, j]) < 1e-14:
            raise ZeroDivisionError(f"Zero pivot at L[{j},{j}] = {L[j,j]}")

        # Row j of U (right of diagonal)
        for i in range(j + 1, n):
            s = 0.0
            for k in range(j):
                s += L[j, k] * U[k, i]
            U[j, i] = (A[j, i] - s) / L[j, j]

    return L, U


def forward_substitution(L: np.ndarray, b: np.ndarray):
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    for i in range(n):
        s = 0.0
        for k in range(i):
            s += L[i, k] * y[k]
        y[i] = (b[i] - s) / L[i, i]
    return y


def back_substitution(U: np.ndarray, y: np.ndarray):
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        s = 0.0
        for k in range(i + 1, n):
            s += U[i, k] * x[k]
        x[i] = (y[i] - s) / U[i, i]
    return x

# Source term for Question 3(b)
def f_source(x, y):
    return -2.0 * math.pi**2 * math.sin(math.pi * x) * math.sin(math.pi * y)

# Assemble A and RHS (2×2 interior)
# Dirichlet boundaries are all zero => no boundary contribution
# Using: 4u - neighbors = -h^2 f
m, n = 2, 2
h = 1.0 / (m + 1)   # h = 1/3 for 2×2 interior

# Unknown order: [u11, u21, u12, u22]^T
A = np.array([
    [ 4.0, -1.0, -1.0,  0.0],
    [-1.0,  4.0,  0.0, -1.0],
    [-1.0,  0.0,  4.0, -1.0],
    [ 0.0, -1.0, -1.0,  4.0]
], dtype=float)

# Interior coordinates: x_i = i*h, y_j = j*h, i,j = 1,2
coords = [(1*h, 1*h), (2*h, 1*h), (1*h, 2*h), (2*h, 2*h)]

rhs = np.zeros(4, dtype=float)
for p, (x, y) in enumerate(coords):
    rhs[p] = -(h**2) * f_source(x, y)

# Solve A x = rhs using Crout LU
L, U = crout_lu(A)
y = forward_substitution(L, rhs)
x = back_substitution(U, y)

u11, u21, u12, u22 = x

np.set_printoptions(precision=10, suppress=True)

print("\n3(c): Poisson 2×2 interior (Crout LU)\n")
print(f"h = {h:.6f}")
print("\nA =\n", A)
print("\nRHS =\n", rhs)

print("\nInterior solution u:")
print(f"u11 = {u11:.10f}")
print(f"u21 = {u21:.10f}")
print(f"u12 = {u12:.10f}")
print(f"u22 = {u22:.10f}")



Question 3(c): Poisson 2×2 interior (Crout LU)

h = 0.333333

A =
 [[ 4. -1. -1.  0.]
 [-1.  4.  0. -1.]
 [-1.  0.  4. -1.]
 [ 0. -1. -1.  4.]]

RHS =
 [1.6449340668 1.6449340668 1.6449340668 1.6449340668]

Interior solution u:
u11 = 0.8224670334
u21 = 0.8224670334
u12 = 0.8224670334
u22 = 0.8224670334


In [ ]:
# Code for 2
import numpy as np

# Crout LU decomposition + forward/back substitution
def crout_lu(A: np.ndarray):
    A = A.astype(float)
    n = A.shape[0]
    L = np.zeros((n, n), dtype=float)
    U = np.zeros((n, n), dtype=float)

    # Crout: diag(U) = 1
    for i in range(n):
        U[i, i] = 1.0

    for j in range(n):
        # Column j of L
        for i in range(j, n):
            s = 0.0
            for k in range(j):
                s += L[i, k] * U[k, j]
            L[i, j] = A[i, j] - s

        if abs(L[j, j]) < 1e-14:
            raise ZeroDivisionError(f"Zero pivot at L[{j},{j}] = {L[j,j]}")

        # Row j of U (right of diagonal)
        for i in range(j + 1, n):
            s = 0.0
            for k in range(j):
                s += L[j, k] * U[k, i]
            U[j, i] = (A[j, i] - s) / L[j, j]

    return L, U


def forward_substitution(L: np.ndarray, b: np.ndarray):
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    for i in range(n):
        s = 0.0
        for k in range(i):
            s += L[i, k] * y[k]
        y[i] = (b[i] - s) / L[i, i]
    return y


def back_substitution(U: np.ndarray, y: np.ndarray):
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        s = 0.0
        for k in range(i + 1, n):
            s += U[i, k] * x[k]
        x[i] = (y[i] - s) / U[i, i]
    return x

# Assemble Laplace A x = b on m×n interior grid
# 5-point stencil: 4u(i,j) - neighbors = boundary contributions
def assemble_laplace_Ab(m, n, T_left, T_right, T_bottom, T_top):
    N = m * n
    A = np.zeros((N, N), dtype=float)
    b = np.zeros(N, dtype=float)

    def idx(i, j):
        # i=1..m, j=1..n
        return (j - 1) * m + (i - 1)

    for j in range(1, n + 1):
        for i in range(1, m + 1):
            p = idx(i, j)

            # Center coefficient
            A[p, p] = 4.0

            # Left neighbor
            if i - 1 >= 1:
                A[p, idx(i - 1, j)] = -1.0
            else:
                b[p] += T_left

            # Right neighbor
            if i + 1 <= m:
                A[p, idx(i + 1, j)] = -1.0
            else:
                b[p] += T_right

            # Bottom neighbor
            if j - 1 >= 1:
                A[p, idx(i, j - 1)] = -1.0
            else:
                b[p] += T_bottom

            # Top neighbor
            if j + 1 <= n:
                A[p, idx(i, j + 1)] = -1.0
            else:
                b[p] += T_top

    return A, b

# Settings for Question 2: 3×3 interior
m, n = 3, 3
T_left, T_right, T_bottom, T_top = 50.0, 25.0, 0.0, 75.0

# Build system
A, b = assemble_laplace_Ab(m, n, T_left, T_right, T_bottom, T_top)

# Solve with Crout LU
L, U = crout_lu(A)
y = forward_substitution(L, b)
x = back_substitution(U, y)

# Reshape solution into 3×3 interior grid (j from 1..n, i from 1..m)
U_interior = x.reshape((n, m))   # rows correspond to j=1..n (bottom to top)

np.set_printoptions(precision=6, suppress=True)

print("\nQuestion 2(a) (coding): Laplace 3×3 interior grid\n")
print("A (9×9) =\n", A)
print("\nb (9×1) =\n", b)

print("\nInterior temperatures (rows: bottom→top, cols: left→right):\n")
print(U_interior)

plate = np.zeros((n + 2, m + 2), dtype=float)
plate[:, 0] = T_left
plate[:, -1] = T_right
plate[0, :] = T_bottom
plate[-1, :] = T_top
plate[1:-1, 1:-1] = U_interior

print("\nFull grid including boundaries (top row first):\n")
print(np.flipud(plate))


Question 2(a) (coding): Laplace 3×3 interior grid

A (9×9) =
 [[ 4. -1.  0. -1.  0.  0.  0.  0.  0.]
 [-1.  4. -1.  0. -1.  0.  0.  0.  0.]
 [ 0. -1.  4.  0.  0. -1.  0.  0.  0.]
 [-1.  0.  0.  4. -1.  0. -1.  0.  0.]
 [ 0. -1.  0. -1.  4. -1.  0. -1.  0.]
 [ 0.  0. -1.  0. -1.  4.  0.  0. -1.]
 [ 0.  0.  0. -1.  0.  0.  4. -1.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.  4. -1.]
 [ 0.  0.  0.  0.  0. -1.  0. -1.  4.]]

b (9×1) =
 [ 50.   0.  25.  50.   0.  25. 125.  75. 100.]

Interior temperatures (rows: bottom→top, cols: left→right):

[[28.571429 21.428571 19.642857]
 [42.857143 37.5      32.142857]
 [55.357143 53.571429 46.428571]]

Full grid including boundaries (top row first):

[[75.       75.       75.       75.       75.      ]
 [50.       55.357143 53.571429 46.428571 25.      ]
 [50.       42.857143 37.5      32.142857 25.      ]
 [50.       28.571429 21.428571 19.642857 25.      ]
 [ 0.        0.        0.        0.        0.      ]]
